In [1]:
# # In case the Int Slider does not appear, run:
# !pip install ipywidgets
# !jupyter lab clean
# # restart kernel

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import matplotlib.colors as mcolors
import plotly.graph_objects as go
import plotly.colors as pc

In [3]:
DISPLAY_EVERYTHING_IN_ONE_PLOT = False

# Define results from which experiment to display

In [4]:
# # 1. To load data from microtrips
# results_subdirectory = "1-For-Microtrips"

In [5]:
# # 2. To load data from full trips 
# results_subdirectory = "2-For-Fulltrips"

In [6]:
# # 3. to load data from full trips (ML trained without calculated columns, only original columns from dataset)
# results_subdirectory = "3-For-Fulltrips-without-calculated-columns-for-ML"

In [7]:
# # 4. to load data from full trips with Battery temperature as feature
# results_subdirectory = "4-For-Full-trips-with-Battery-Temperature"

In [8]:
# # 5. To load data from full trips (without early stpooing, learning rate = 1e-4)
# results_subdirectory = "5-For-Full-Trips-without-EarlyStopping_LearningRate-1e-4"

In [9]:
# 6. To load data from full trips (without early stpooing, learning rate = 1e-3)
results_subdirectory = "6-For-Full-Trips-without-EarlyStopping_LearningRate-1e-3"

In [10]:
model_names = [
    "-only-MachineLearning",
    "Physics-only",
    "Physics-and-MachineLearning",
]

# add missing folder names to model paths
models_results_paths = [f"results/{results_subdirectory}/{model_name}/test_data_and_model_output" for model_name in model_names]

models_results_paths

['results/6-For-Full-Trips-without-EarlyStopping_LearningRate-1e-3/-only-MachineLearning/test_data_and_model_output',
 'results/6-For-Full-Trips-without-EarlyStopping_LearningRate-1e-3/Physics-only/test_data_and_model_output',
 'results/6-For-Full-Trips-without-EarlyStopping_LearningRate-1e-3/Physics-and-MachineLearning/test_data_and_model_output']

# Get data

In [11]:
# data structure: Each set of results will be stored as one element of a list (all_models_results).
# Each element will contain all the test data and model output files
all_models_results_filepaths = []

In [12]:
def get_filepaths_with_extension(file_extension=".csv", directory="."):
    output = []
    
    for file in os.listdir(directory):  
        # Check if the file has the required extension
        if file.endswith(file_extension):
            output.append(f"{directory}/{file}") 
            
    return output

In [13]:
# get filepaths of all test results of all models
for path in models_results_paths:
    model_result_datafile_paths = get_filepaths_with_extension(file_extension=".csv", directory=f"{path}")

    all_models_results_filepaths.append(model_result_datafile_paths)

In [14]:
all_models_results_filepaths[:2]

[['results/6-For-Full-Trips-without-EarlyStopping_LearningRate-1e-3/-only-MachineLearning/test_data_and_model_output/file-0.csv',
  'results/6-For-Full-Trips-without-EarlyStopping_LearningRate-1e-3/-only-MachineLearning/test_data_and_model_output/file-1.csv',
  'results/6-For-Full-Trips-without-EarlyStopping_LearningRate-1e-3/-only-MachineLearning/test_data_and_model_output/file-2.csv',
  'results/6-For-Full-Trips-without-EarlyStopping_LearningRate-1e-3/-only-MachineLearning/test_data_and_model_output/file-3.csv',
  'results/6-For-Full-Trips-without-EarlyStopping_LearningRate-1e-3/-only-MachineLearning/test_data_and_model_output/file-4.csv',
  'results/6-For-Full-Trips-without-EarlyStopping_LearningRate-1e-3/-only-MachineLearning/test_data_and_model_output/file-5.csv',
  'results/6-For-Full-Trips-without-EarlyStopping_LearningRate-1e-3/-only-MachineLearning/test_data_and_model_output/file-6.csv',
  'results/6-For-Full-Trips-without-EarlyStopping_LearningRate-1e-3/-only-MachineLearning/

In [15]:
# read all model results
# each model will be represented by an index, and inside that index, all the dfs that contain the test results
lst_models_dfs = []    

for idx_model in range(len(all_models_results_filepaths)):
    model_dfs = [pd.read_csv(filepath, sep=";", encoding="ISO-8859-2") for filepath in all_models_results_filepaths[idx_model]]
    lst_models_dfs.append(model_dfs)

# Create interactive plots

In [16]:
# !pip install plotly

In [17]:
# if DISPLAY_EVERYTHING_IN_ONE_PLOT = False, display each model in a different plot
if not DISPLAY_EVERYTHING_IN_ONE_PLOT:
    def plot_soc(file_id, model_outputs_to_display):
        
        # Use Plotly's "Safe" qualitative color palette. Source of color pallete: https://plotly.com/python/discrete-color/
        colorblind_palette = pc.qualitative.Safe
        
        # Loop through each model result
        for idx_model in range(len(lst_models_dfs)):
            # Get column name of model output. Model output is the last column
            model_output_column_name = lst_models_dfs[idx_model][file_id].columns[-1]
            
            # If model_output_column_name is not in selected model_outputs_to_display, skip
            if model_output_column_name not in model_outputs_to_display:
                continue
            
            # Create a new figure for each model
            fig = go.Figure()
            
            # Plot actual SoC
            fig.add_trace(go.Scatter(
                x=lst_models_dfs[idx_model][file_id]['Time [s]'],
                y=lst_models_dfs[idx_model][file_id]['SoC [%]'],
                mode='lines',
                line=dict(width=3, color=colorblind_palette[0]),
                name='Actual SoC [%]'
            ))
            
            # Plot the model output
            fig.add_trace(go.Scatter(
                x=lst_models_dfs[idx_model][file_id]['Time [s]'],
                y=lst_models_dfs[idx_model][file_id][model_output_column_name],
                mode='lines',
                line=dict(dash='dot', width=3, color=colorblind_palette[(idx_model % len(colorblind_palette)) + 1]),
                name=f"Model: {model_output_column_name}"
            ))
            
            # Update layout
            fig.update_layout(
                title=f'Actual vs Estimated State of Charge (SoC %) - Model: {model_output_column_name} - File {file_id}',
                xaxis_title='Time [s]',
                yaxis_title='State of Charge [%]',
                legend_title='Legend',
                template='plotly_white'
            )
            
            # Show the plot
            fig.show()


In [18]:
if DISPLAY_EVERYTHING_IN_ONE_PLOT:
    def plot_soc(file_id, model_outputs_to_display):
        
        # Use Plotly's "Safe" qualitative color palette. Source of color pallete: https://plotly.com/python/discrete-color/
        colorblind_palette = pc.qualitative.Safe
        
        # Create the figure
        fig = go.Figure()
        
        # Plot actual SoC only from first model
        fig.add_trace(go.Scatter(
            x=lst_models_dfs[0][file_id]['Time [s]'],
            y=lst_models_dfs[0][file_id]['SoC [%]'],
            mode='lines',
            line=dict(width=3, color=colorblind_palette[0]),
            name='Actual SoC [%]'
        ))
        
        # Plot the model result of each model
        for idx_model in range(len(lst_models_dfs)):
            # Get column name of model output. Model output is the last column
            model_output_column_name = lst_models_dfs[idx_model][file_id].columns[-1]
            
            # If model_output_column_name is not in selected model_outputs_to_display, skip
            if model_output_column_name not in model_outputs_to_display:
                continue
            
            fig.add_trace(go.Scatter(
                x=lst_models_dfs[idx_model][file_id]['Time [s]'],
                y=lst_models_dfs[idx_model][file_id][model_output_column_name],
                mode='lines',
                line=dict(dash='dot', width=3, color=colorblind_palette[(idx_model % len(colorblind_palette)) + 1]),
                name=f"Model: {model_output_column_name}"
            ))
        
        # Update layout
        fig.update_layout(
            title=f'Actual vs Estimated State of Charge (SoC %) of file {file_id}',
            xaxis_title='Time [s]',
            yaxis_title='State of Charge [%]',
            legend_title='Legend',
            template='plotly_white'
        )
        
        # Show the plot
        fig.show()

In [19]:
# get model names
model_output_from_dfs = [ model_df[0].columns[-1] for model_df in lst_models_dfs ]
model_output_from_dfs

['-only-MachineLearning', 'Physics-only-', 'Physics-and-MachineLearning']

In [20]:
# Create Slider for multiple selection of models
model_select_multiple = widgets.SelectMultiple(
    options=model_output_from_dfs,
    value=model_output_from_dfs,
    rows=len(model_output_from_dfs),
    description='Models:',
    disabled=False
)

# Create a slider for selecting the ID
id_slider = widgets.IntSlider(
    value=0, 
    min=0, 
    max=(len(lst_models_dfs[0]) -1),    # max is the total number of file in any of the model results 
    step=1, 
    description='Test file ID:',
    continuous_update=False,
    style={'slider_button_width': '20px'}  # Enable built-in buttons
)

# Create a horizontal layout for the id_slider and model_select_multiple
controls = widgets.HBox([model_select_multiple, id_slider])

# Connect the slider to the plot function
interactive_plot = widgets.interactive_output(plot_soc, {'file_id': id_slider, 'model_outputs_to_display': model_select_multiple})

# Display the controls widgets and plot together
display(controls, interactive_plot)

Output()